**Load Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/spam (1).csv", encoding="latin-1")

dataset = dataset.dropna(how="any", axis=1)
dataset.columns = ['target', 'message']

dataset.head()

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
dataset['message_len'] = dataset['message'].apply(lambda x: len(x.split(' ')))
dataset.head()

,target,message,message_len
0,ham,"Go until jurong point, crazy.. Available only ...",20
1,ham,Ok lar... Joking wif u oni...,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28
3,ham,U dun say so early hor... U c already then say...,11
4,ham,"Nah I don't think he goes to usf, he lives aro...",13


In [ ]:
max(dataset['message_len'])

171

**Exploratory Data Analysis**

In [ ]:
balance_counts = dataset.groupby('target')['target'].agg('count').values
balance_counts

array([4825,  747])

In [ ]:
import plotly.graph_objs as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['ham'],
    y=[balance_counts[0]],
    name='ham',
    text=[balance_counts[0]],
    textposition='auto',
   
))
fig.add_trace(go.Bar(
    x=['spam'],
    y=[balance_counts[1]],
    name='spam',
    text=[balance_counts[1]],
    textposition='auto',
    
))
fig.update_layout(
    title='<span style="font-size:32px; font-family:Times New Roman">Dataset distribution by target</span>'
)
fig.show()

In [ ]:
ham_dataset = dataset[dataset['target'] == 'ham']['message_len'].value_counts().sort_index()
spam_dataset = dataset[dataset['target'] == 'spam']['message_len'].value_counts().sort_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ham_dataset.index,
    y=ham_dataset.values,
    name='ham',
    fill='tozeroy',
    
))
fig.add_trace(go.Scatter(
    x=spam_dataset.index,
    y=spam_dataset.values,
    name='spam',
    fill='tozeroy',
    
))
fig.update_layout(
    title='<span style="font-size:32px; font-family:Times New Roman">Data Roles in Different Fields</span>'
)
fig.update_xaxes(range=[0, 70])
fig.show()

**Data Preprocessing**

Case Folding

In [ ]:
import re 
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
dataset['message_clean'] = dataset['message'].apply(clean_text)
dataset.head()

,target,message,message_len,message_clean
0,ham,"Go until jurong point, crazy.. Available only ...",20,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,6,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,free entry in a wkly comp to win fa cup final...
3,ham,U dun say so early hor... U c already then say...,11,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,"nah i don't think he goes to usf, he lives aro..."


Tokenisasi

In [ ]:
def tokenize(message_clean):
    token = nltk.word_tokenize(message_clean)
    return token

In [ ]:

import nltk
nltk.download('punkt')
dataset['tokenized'] = dataset['message_clean'].apply(tokenize)

print('Tokenizing Result : \n') 
print(dataset['tokenized'].head())
print('\n\n\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Tokenizing Result : 

0    [go, until, jurong, point, ,, crazy, .., avail...
1             [ok, lar, ..., joking, wif, u, oni, ...]
2    [free, entry, in, a, wkly, comp, to, win, fa, ...
3    [u, dun, say, so, early, hor, ..., u, c, alrea...
4    [nah, i, do, n't, think, he, goes, to, usf, ,,...
Name: tokenized, dtype: object






Stopwords

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stop_words(message_clean):
    removed_stop_words = []
    for review in message_clean:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in stop_words])
        )
    return removed_stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dataset['stopwords_removed'] = dataset['tokenized'].apply(remove_stop_words)

print('Stopwords Result : \n') 
print(dataset['stopwords_removed'].head())
print('\n\n\n')

Stopwords Result : 

0    [go, , jurong, point, ,, crazy, .., available,...
1             [ok, lar, ..., joking, wif, u, oni, ...]
2    [free, entry, , , wkly, comp, , win, fa, cup, ...
3    [u, dun, say, , early, hor, ..., u, c, already...
4    [nah, , , n't, think, , goes, , usf, ,, , live...
Name: stopwords_removed, dtype: object






Stemming

In [ ]:
dataset['stemming_removed'] = dataset['stopwords_removed'].apply(remove_stop_words)

print('Stemming Result : \n') 
print(dataset['stopwords_removed'].head())
print('\n\n\n')

Stemming Result : 

0    [go, , jurong, point, ,, crazy, .., available,...
1             [ok, lar, ..., joking, wif, u, oni, ...]
2    [free, entry, , , wkly, comp, , win, fa, cup, ...
3    [u, dun, say, , early, hor, ..., u, c, already...
4    [nah, , , n't, think, , goes, , usf, ,, , live...
Name: stopwords_removed, dtype: object






Vectorization 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(dataset['target'])

dataset['target_encoded'] = le.transform(dataset['target'])
dataset.head()

,target,message,message_len,message_clean,tokenized,stopwords_removed,stemming_removed,target_encoded
0,ham,"Go until jurong point, crazy.. Available only ...",20,"go until jurong point, crazy.. available only ...","[go, until, jurong, point, ,, crazy, .., avail...","[go, , jurong, point, ,, crazy, .., available,...","[go, , jurong, point, ,, crazy, .., available,...",0
1,ham,Ok lar... Joking wif u oni...,6,ok lar... joking wif u oni...,"[ok, lar, ..., joking, wif, u, oni, ...]","[ok, lar, ..., joking, wif, u, oni, ...]","[ok, lar, ..., joking, wif, u, oni, ...]",0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,free entry in a wkly comp to win fa cup final...,"[free, entry, in, a, wkly, comp, to, win, fa, ...","[free, entry, , , wkly, comp, , win, fa, cup, ...","[free, entry, , , wkly, comp, , win, fa, cup, ...",1
3,ham,U dun say so early hor... U c already then say...,11,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, ..., u, c, alrea...","[u, dun, say, , early, hor, ..., u, c, already...","[u, dun, say, , early, hor, ..., u, c, already...",0
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,"nah i don't think he goes to usf, he lives aro...","[nah, i, do, n't, think, he, goes, to, usf, ,,...","[nah, , , n't, think, , goes, , usf, ,, , live...","[nah, , , n't, think, , goes, , usf, ,, , live...",0


In [ ]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
x = dataset['message_clean']
y = dataset['target_encoded']

print(len(x), len(y))

5572 5572


In [ ]:
# Split into train and test sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

4179 4179
1393 1393


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x_train)

CountVectorizer()

In [ ]:
# Use the trained to create a document-term matrix from train and test sets
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)

Word Embeding : GlOVE

In [ ]:
texts = dataset['message_clean']
target = dataset['target_encoded']

In [ ]:
# Calculate the length of our vocabulary
from keras.preprocessing.text import Tokenizer
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

7736

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-06-16 04:44:15--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-16 04:44:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-16 04:44:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
import numpy as np
embeddings_dictionary = dict()
embedding_dim = 100

# Load GloVe 100D embeddings
with open('/content/glove.6B.100d.txt') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions

# embeddings_dictionary

In [ ]:
embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.046539  ,  0.61966002,  0.56647003, ..., -0.37616   ,
        -0.032502  ,  0.80620003],
       [-0.18970001,  0.050024  ,  0.19084001, ..., -0.39804   ,
         0.47646999, -0.15983   ],
       ...,
       [-0.12201   ,  0.20465   ,  0.53192002, ..., -0.164     ,
        -0.41637   ,  0.66426003],
       [-0.41600999,  0.16974001,  0.43457001, ..., -0.083432  ,
         0.49520999,  0.77258003],
       [ 0.46118999,  0.3567    ,  0.026078  , ..., -0.0056525 ,
         0.55176002,  0.77563   ]])

TF-IDF 

In [ ]:
def calc_TF(message_clean):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in message_clean:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(target)
    return TF_dict

dataset["embeddings_dictionary"] = dataset['stemming_removed'].apply(calc_TF)

dataset["embeddings_dictionary"].head()

0    {'go': 0.00017946877243359656, '': 0.000717875...
1    {'ok': 0.00017946877243359656, 'lar': 0.000179...
2    {'free': 0.00017946877243359656, 'entry': 0.00...
3    {'u': 0.0003589375448671931, 'dun': 0.00017946...
4    {'nah': 0.00017946877243359656, '': 0.00107681...
Name: embeddings_dictionary, dtype: object

In [70]:
index = 2

print('%20s' % "term", "\t", "TF\n")
for key in dataset["embeddings_dictionary"][index]:
    print('%20s' % key, "\t", dataset["embeddings_dictionary"][index][key])

                term 	 TF

                free 	 0.00017946877243359656
               entry 	 0.0003589375448671931
                     	 0.0010768126346015793
                wkly 	 0.00017946877243359656
                comp 	 0.00017946877243359656
                 win 	 0.00017946877243359656
                  fa 	 0.0003589375448671931
                 cup 	 0.00017946877243359656
               final 	 0.00017946877243359656
                tkts 	 0.00017946877243359656
                 may 	 0.00017946877243359656
                   . 	 0.00017946877243359656
                text 	 0.00017946877243359656
             receive 	 0.00017946877243359656
            question 	 0.00017946877243359656
                   ( 	 0.00017946877243359656
                 std 	 0.00017946877243359656
                 txt 	 0.00017946877243359656
                rate 	 0.00017946877243359656
                   ) 	 0.00017946877243359656
                   & 	 0.00017946877243359656
          

In [71]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(dataset["embeddings_dictionary"])
DF

{'go': 260,
 '': 5345,
 'jurong': 1,
 'point': 13,
 ',': 1280,
 'crazy': 13,
 '..': 454,
 'available': 17,
 'bugis': 7,
 'n': 121,
 'great': 107,
 'world': 35,
 'la': 7,
 'e': 73,
 'buffet': 2,
 '...': 719,
 'cine': 7,
 'got': 238,
 'amore': 1,
 'wat': 102,
 'ok': 244,
 'lar': 37,
 'joking': 5,
 'wif': 26,
 'u': 803,
 'oni': 4,
 'free': 226,
 'entry': 21,
 'wkly': 14,
 'comp': 11,
 'win': 67,
 'fa': 2,
 'cup': 8,
 'final': 18,
 'tkts': 4,
 'may': 47,
 '.': 2610,
 'text': 170,
 'receive': 34,
 'question': 21,
 '(': 115,
 'std': 9,
 'txt': 159,
 'rate': 24,
 ')': 398,
 '&': 475,
 'c': 109,
 "'s": 440,
 'apply': 32,
 'dun': 51,
 'say': 87,
 'early': 31,
 'hor': 2,
 'already': 89,
 'nah': 10,
 "n't": 294,
 'think': 129,
 'goes': 25,
 'usf': 10,
 'lives': 4,
 'around': 57,
 'though': 27,
 'freemsg': 14,
 'hey': 105,
 'darling': 5,
 'week': 105,
 'word': 38,
 'back': 147,
 '!': 924,
 "'d": 27,
 'like': 227,
 'fun': 30,
 'still': 152,
 '?': 1216,
 'tb': 4,
 'xxx': 26,
 'chgs': 1,
 'send': 186

In [72]:
import numpy as np
n_document = len(dataset)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [73]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
dataset["TF-IDF_dict"] = dataset["embeddings_dictionary"].apply(calc_TF_IDF)

In [74]:
# Check TF-IDF result
index = 2

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in dataset["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", dataset["embeddings_dictionary"][index][key] ,"\t" , dataset["TF-IDF_dict"][index][key])
    
TF_IDF = pd.DataFrame(dataset, columns=['term', 'rank'])

                term 	         TF 	              TF-IDF

                free 	 0.00017946877243359656 	 0.0005744004517979709
               entry 	 0.0003589375448671931 	 0.0019865279546092537
                     	 0.0010768126346015793 	 4.458589899587071e-05
                wkly 	 0.00017946877243359656 	 0.0010619991266686084
                comp 	 0.00017946877243359656 	 0.0011020464258994429
                 win 	 0.00017946877243359656 	 0.0007907397038269185
                  fa 	 0.0003589375448671931 	 0.0027016859462424933
                 cup 	 0.00017946877243359656 	 0.001153676374293517
               final 	 0.00017946877243359656 	 0.0010195747228523432
                tkts 	 0.00017946877243359656 	 0.0012591657254963382
                 may 	 0.00017946877243359656 	 0.0008532498786776392
                   . 	 0.00017946877243359656 	 0.00013604105638919016
                text 	 0.00017946877243359656 	 0.0006252415251968838
             receive 	 0.00017946877

In [75]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

dataset["TF_IDF_Vec"] = dataset["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(dataset["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(dataset["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[2.9723932663913805e-05, 0.0, 0.00026383938137167304, 0.0, 0.0, 0.0, 0.0007344788667945425, 0.0, 0.0, 0.0, 0.0004496073072413399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0005493519252650759, 0.0, 0.0, 0.0, 0.0, 0.0005651553810064942, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


In [76]:
# Convert Series to List
TF_IDF_Vec_List = np.array(dataset["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
1,.,0.662112
2,",",0.503406
4,!,0.444739
3,?,0.420478
6,...,0.415348
9,&,0.404423
5,u,0.389128
15,;,0.381845
10,..,0.315624
7,:,0.298980


Cosine Similarity 

In [77]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

#define arrays
texts = np.random.randint(10, size=100)
target = np.random.randint(10, size=100)

#calculate Cosine Similarity
cos_sim = dot(texts, target)/(norm(texts)*norm(target))

cos_sim



0.6973995059369927

Example 

In [78]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

#define arrays
dataset.stemming_removed[1] = np.random.randint(10, size=100)
dataset.stemming_removed[2] = np.random.randint(10, size=100)

#calculate Cosine Similarity
cos_sim = dot(dataset.stemming_removed[1], dataset.stemming_removed[2])/(norm(dataset.stemming_removed[1])*norm(dataset.stemming_removed[2]))





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
cos_sim

0.7621901106562916

Evaluation 

In [80]:
# Create a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(x_train_dtm, y_train)

MultinomialNB()

In [81]:
# Make class anf probability predictions
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

In [82]:
# calculate accuracy of class predictions
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))


0.9777458722182341
